# 포트폴리오의 국가별 매출분포 계산

1. 포트폴리오 및 georev 호출
2. 합병
3. 피벗

In [48]:
import pandas as pd
import plotly.express as px

In [49]:
weights = pd.read_excel('pf_weights.xlsx')

In [50]:
geo = pd.read_excel('geo_revenue.xlsx')

In [51]:
weights.columns = weights.columns.astype(str)

In [52]:
weights

,Unnamed: 0,2014-01-08 00:00:00,2014-04-20 00:00:00,2014-11-10 00:00:00
0,TEF-ES,0.05,0.08,0.05
1,ARM-GB,0.07,0.07,0.10
2,ML-FR,0.09,0.06,0.09
3,DUE-DE,0.10,0.04,0.10
4,RWE-DE,0.12,0.06,0.04
5,VIS-ES,0.15,0.12,0.15
6,SOON-CH,0.02,0.10,0.02
7,SU-FR,0.10,0.17,0.10
8,AAPL-US,0.30,0.30,0.35


In [53]:
# Unnamed:0 를 ticker로 변경, ticker(id_vars)를 제외한 모든 칼럼 melting.

weights_melt = weights.rename({
  'Unnamed: 0': 'ticker'  
}, axis=1).melt(id_vars='ticker', value_vars=['2014-01-08 00:00:00', '2014-04-20 00:00:00', '2014-11-10 00:00:00'], var_name='Periods', value_name='weights')

In [54]:
# Periods 칼럼 datetime

weights_melt['Periods'] = pd.to_datetime(weights_melt['Periods'].apply(lambda x: x[:10]))

In [55]:
weights_melt

,ticker,Periods,weights
0,TEF-ES,2014-01-08,0.05
1,ARM-GB,2014-01-08,0.07
2,ML-FR,2014-01-08,0.09
3,DUE-DE,2014-01-08,0.10
4,RWE-DE,2014-01-08,0.12
5,VIS-ES,2014-01-08,0.15
6,SOON-CH,2014-01-08,0.02
7,SU-FR,2014-01-08,0.10
8,AAPL-US,2014-01-08,0.30
9,TEF-ES,2014-04-20,0.08


In [56]:
geo_filtered = geo[geo['layer_number'] == 1]

In [57]:
geo_filtered = geo_filtered[['ticker', 'latest_fiscal_yearend', 'region_name', 'est_percent']]

In [58]:
weights_melt = weights_melt.sort_values(['Periods'])
geo_filtered = geo_filtered.sort_values(['latest_fiscal_yearend'])

In [59]:
merged_df = pd.merge_asof(weights_melt, geo_filtered, left_on='Periods', right_on='latest_fiscal_yearend', by='ticker', direction='backward')

In [60]:
merged_df['est_percent_weighted'] = merged_df['est_percent'] *  merged_df['weights']

In [61]:
merged_df

,ticker,Periods,weights,latest_fiscal_yearend,region_name,est_percent,est_percent_weighted
0,TEF-ES,2014-01-08,0.05,2013-12-31,Europe,48.725739,2.436287
1,ARM-GB,2014-01-08,0.07,2013-12-31,Asia/Pacific,51.297715,3.590840
2,ML-FR,2014-01-08,0.09,2013-12-31,Asia/Pacific,3.594199,0.323478
3,DUE-DE,2014-01-08,0.10,2013-12-31,Asia/Pacific,34.514588,3.451459
4,RWE-DE,2014-01-08,0.12,2013-12-31,Asia/Pacific,0.423588,0.050831
5,VIS-ES,2014-01-08,0.15,2013-12-31,Asia/Pacific,17.088602,2.563290
6,SOON-CH,2014-01-08,0.02,NaT,NaN,NaN,NaN
7,SU-FR,2014-01-08,0.10,2013-12-31,Americas,31.260673,3.126067
8,AAPL-US,2014-01-08,0.30,NaT,NaN,NaN,NaN
9,AAPL-US,2014-04-20,0.30,NaT,NaN,NaN,NaN


In [62]:
grouped = merged_df[['Periods', 'region_name', 'est_percent_weighted']].groupby(['Periods', 'region_name']).sum('est_percent_weighted')

In [63]:
grouped = grouped.reset_index()

In [64]:
grouped['est_percent_weighted']

0     3.126067
1     9.979898
2     2.436287
3     5.314314
4     7.263123
5     7.119315
6     3.126067
7    11.484942
8    10.225171
Name: est_percent_weighted, dtype: float64

In [65]:
import matplotlib.pyplot as plt
import seaborn as SNS

In [66]:
SNS.lineplot(grouped, x='Periods', y='est_percent_weighted', hue='region_name')

c:\Users\jchoi02\Anaconda3\lib\site-packages\seaborn\_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(


ValueError: Could not interpret value `est_percent_weighted` for parameter `y`